#Import Libraries


In [25]:
import re
import json
from math import *
import numpy as np
import pandas as pd

In [26]:
!git clone https://github.com/sheffieldnlp/fever-baselines.git

Cloning into 'fever-baselines'...
remote: Enumerating objects: 3092, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 3092 (delta 13), reused 1 (delta 0), pack-reused 3062
Receiving objects: 100% (3092/3092), 1.04 MiB | 22.56 MiB/s, done.
Resolving deltas: 100% (2184/2184), done.


In [27]:
!ls

config	    fever-baselines  README.md	       scripts
Dockerfile  LICENSE	     requirements.txt  src


In [28]:
%cd /content/fever-baselines

/content/fever-baselines


In [29]:
!ls

config	    fever-baselines  README.md	       scripts
Dockerfile  LICENSE	     requirements.txt  src


In [30]:
import sys
sys.path.append('/content/fever-baselines')

#Importing data

In [31]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [33]:
PATH = '/content/gdrive/My Drive/project_work/'

In [34]:
def similar(a, b, norm=False): #defining cosine similarity
    assert len(a) == len(b)
    given_list = [0] * len(b)
    if a == given_list or b == given_list:
        return float(1) if a == b else float(0)
    
    for i in range(len(a)):  
       x = np.array([a[i] * b[i], a[i] * a[i], b[i] * b[i]])
    y = sum(x[:, 0]) / (np.sqrt(sum(x[:, 1])) * np.sqrt(sum(x[:, 2])))
    
    return 0.5 * y + 0.5 if norm else y

#Extraction from wikipedia

In [35]:
def wikip(wiki_dir): 
    dicted = dict()
    for i in range(1,110):
        jnum="{:03d}".format(i)
        fname=wiki_dir+"wiki-"+jnum+".jsonl"
        f=open(fname)
        sen=f.readline()
        while sen:
          dt=json.loads(sen.rstrip("\n"))
          doc_id=dt["id"]
          tt = dt["text"]
          if tt != "":
              dicted[doc_id]=tt
          sen=f.readline()
    np.save(PATH + "my_task2.npy",dicted)
    print("It is completed")

#Create Dictionary

In [19]:
def create_dictionary(dicted): #Create dictionary
    dt = np.load(dicted, allow_pickle=True).item()
    dictionary = {}
    number_ducument = len(dt)
    for d in dt.items():
        text = list(set(d[1].split(' ')))[1:]
        for i in text:
            i = re.sub("[,.。:_=+*&^%$#@!?()<>/`';|]", "", i)
            if i.isdigit():
                continue
            if not i in dictionary:
                dictionary[i] = [d[0]]
            else:
                dictionary[i].append(d[0])
    np.save(PATH + "my_task2.npy",dictionary)
    print(number_ducument)

#TF-IDF terms in the claim

In [21]:
def Subtask2_cossim(claim_id, numberofducuments):
    data = np.load(PATH + 'my_task2.npy', allow_pickle=True).item()
    dictory = np.load(PATH + 'my_task2.npy', allow_pickle=True).item()
    td = np.load('/content/fever-baselines/config/test.json')

    clm = None
    for d in td:
        if d['id'] == claim_id:
            d['claim'] = re.sub("[,.。:_=+*&^%$#@!?()<>/`';|]", "", d['claim'])
            clm = d['claim'].split(' ')
            break
    print(d['id'])
    print(d['claim'])

    claim_tfidf = []
    ky = []
    for i in clm:
        tf = clm.count(i) / len(clm)
        idf = log((numberofducuments / (1 + (len(dictory[i]) if i in dictory else 0))))
        ky.append((i, tf * idf, idf, tf))
    ky.sort(key=lambda x: x[1], reverse=True)
    ky = ky[:5]
    word = [k[0] for k in ky]
    vec1 = [k[1] for k in ky]
    print(word)
    print(vec1)
    
    doc_tf_idf = []
    for d in data.items():
        tt = d[1].split(' ')
        vec2 = []
        for k in ky:
            tf = tt.count(k[0]) / len(tt)
            idf = k[2]
            vec2.append(tf * idf)
        sm= similar(vec1, vec2)
        doc_tf_idf.append([d[0], sm])
    doc_tf_idf.sort(key=lambda x: x[1], reverse=True)
    return doc_tf_idf[:5]

#Calling all funcations

In [ ]:
wikip('/content/gdrive/MyDrive/project_work/data/wiki-pages/')

In [ ]:
create_dictionary(PATH + 'my_task2.npy')

In [ ]:
index_list = [75397,150448,214861]
length_of_ducuments = 5396106

In [ ]:
for index in index_list:
    print(Subtask2_cossim(index, length_of_ducuments))